# Import

## Libraries

In [1]:
# Warning control
import warnings
import os
import yaml
from pathlib import Path


from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool

from pydantic import BaseModel, Field
from typing import List, Optional
from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

## Conversation Transcript

Output from `pdf_to_md.ipynb`:

In [2]:
transcript_content = Path("data/conversation_01.txt").read_text()
print(transcript_content[:500])

CSR: Good morning! Thank you for calling ShiokAh Insurance. My name is Sarah. How may I assist you today?

Customer: Hi Sarah! I want to check about travel insurance leh. Planning to go Japan next month.

CSR: I'll be happy to help you with that. Before we look into the options, could you share some information with me? May I know your age?

Customer: I'm 62 this year.

CSR: Thank you for sharing. Since you're above 60, we'll need to note this for coverage considerations. Do you have any pre-exi


## Tasks & Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [4]:
print(
    "agents_config contains:",
    ", ".join(agents_config.keys()),
    "\ntasks_config contains:",
    ", ".join(tasks_config.keys()),
)

agents_config contains: conversation_transcript_agent, underwriting_report_agent 
tasks_config contains: process_transcript_task, generate_underwriting_report_task


## Tools (Agent Usage)

In [5]:
read_conversation = FileReadTool(file_path="./data/conversation_01.txt")
read_income = FileReadTool(file_path="./data/income_travel_insurance_2024.txt")
read_ge = FileReadTool(file_path="data/ge_travel_insurance_2024.txt")

# Pydantic Classes

In [6]:
class CustomerInfo(BaseModel):
    age: int
    medical_conditions: Optional[List[str]] = Field(default_factory=list)
    destination: str
    coverage_factors: List[str] = Field(default_factory=list)
    policy_customization_factors: List[str] = Field(default_factory=list)


class CustomerInfoList(BaseModel):
    customers: List[CustomerInfo] = Field(default_factory=list)

# Create Crew, Agents and Tasks

In [8]:
conversation_transcript_agent = Agent(
    config=agents_config["conversation_transcript_agent"],
    tools=[read_conversation],
    verbose=True,
)

underwriting_report_agent = Agent(
    config=agents_config["underwriting_report_agent"],
    tools=[read_income],
    verbose=True,
)

process_transcript_task = Task(
    config=tasks_config["process_transcript_task"],
    agent=conversation_transcript_agent,
    output_pydantic=CustomerInfoList,
)

generate_underwriting_report_task = Task(
    config=tasks_config["generate_underwriting_report_task"],
    context=[process_transcript_task],
    agent=underwriting_report_agent,
    async_execution=True,
)

In [9]:
recsys_crew = Crew(
    agents=[conversation_transcript_agent, underwriting_report_agent],
    tasks=[
        process_transcript_task,
        generate_underwriting_report_task,
    ],
    verbose=True,
)

In [10]:
result = recsys_crew.kickoff()
display(Markdown(result.raw))


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Task: This task involves analyzing a conversation transcript between a potential customer interested in travel insurance and a customer service representative from ShiokAh Insurance. The goal is to extract key customer details from the transcript, such as age, medical conditions, destination, coverage preferences, and customization requests. This extracted information will be organized into a Python list and used by an underwriter to determine policy suitability.


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: To complete the task of extracting key customer details from the conversation transcript, I need to read the content of the transcript first.
## Using tool: Read a file's content
## Tool Input: 
"{}"
## Tool Output: 
CSR: Good morning! Thank you for calling ShiokAh Insurance. My name is Sarah. How may I assist you today?

Customer: Hi Sarah! I want to check about travel insurance leh. Plannin

```markdown
# Policy Summary

This underwriting report provides an analysis based on customer information in relation to the travel insurance policy details.

## Age

- **Customer Age:** 62
- **Spouse Age:** 60
- **Considerations:** 
  - Since both travelers are above 60, age-related considerations may apply, such as increased premiums or exclusions related to age.
  - Example from transcript: The CSR noted the customer's age over 60 for coverage considerations.

## Medical Conditions

- **Conditions:** High blood pressure, diabetes (controlled with medication)
- **Considerations:**
  - Potential exclusions or premium adjustments for pre-existing conditions.
  - No hospitalizations in the past 12 months.
  - Regular checkups every 3 months.
  - Example from transcript: The customer mentioned both medical conditions are under control without recent hospitalizations.

## Destination

- **Destination:** Japan
- **Trip Duration:** 2 weeks (15th December to 29th December)
- **Considerations:** 
  - Winter travel considerations such as weather-related delays or health issues.
  - Example from transcript: The customer plans to visit Mount Fuji and engage in light hiking.

## Coverage Factors

- **Medical Emergencies Coverage Preferences:**
  - Emergency medical evacuation
  - Hospital daily cash benefit
  - Outpatient treatment
- **Additional Coverage Preferences:**
  - Flight delays
  - Trip cancellation
- **Considerations:**
  - Priority for medical evacuation coverage in rural areas.
  - Potential premium adjustments for comprehensive coverage needs.
  - Example from transcript: The customer emphasized the importance of evacuation coverage if anything happens in rural areas.

## Policy Customization Factors

- **Excess Preference:** Lower excess
- **Considerations:**
  - Customer prefers lower excess for peace of mind, which may result in higher premiums.
  - Example from transcript: The customer indicated a preference for a lower excess to avoid financial surprises when making claims.

This report outlines factors to be considered when underwriting the travel policy for the customer and their spouse, focusing on potential premium impacts and necessary coverage adjustments tailored to the customer's stated needs and preferences.
```

This report is designed to provide a structured analysis to aid in underwriting decisions, focusing on the customer's specific circumstances and preferences in comparison to policy allowances and adjustments.